In [1]:
import pandas as pd

import json
import glob
import os
import re

In [2]:
base = "../exp/"
paths = glob.glob(base + "preprocessing/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
def path_to_graph(path):
    return {
        'ger06': 'TDGer06',
        'ptv17': 'TDEur17',
        'ptv20': 'TDEur20',
        'osm_europe': 'OSM Europe',
        'osm_ger': 'OSM Ger',
        'osm_ger_td': 'TD OSM Ger',
        'europe': 'DIMACs Europe',
    }[[x for x in path.split('/') if x != ''][-1]]

graphs = pd.DataFrame.from_records([{
    **run,
    'graph': path_to_graph(run['args'][1]),
    'num_nodes': run['graph']['num_nodes'],
    'num_edges': run['graph']['num_arcs'],
} for run in data if 'graph' in run])

In [4]:
runtime_pattern = re.compile(".*Needed (\\d+)musec\\..*")

def parse_contraction_output(path):
  stats = { 'ch_contraction_running_time_s': 0.0 }

  with open(path, 'r') as f:
    for line in f:
      if not 'graph' in stats:
        stats['graph'] = path_to_graph(line.strip())
      else:
        match = runtime_pattern.match(line)
        if match:
          stats['ch_contraction_running_time_s'] += int(match[1]) / 1000000

  return stats

ch_preprocessing = pd.DataFrame.from_records([parse_contraction_output(path) for path in glob.glob(base + "preprocessing/ch/*.out")])

In [5]:
runtime_pattern = re.compile(".*running time : (\\d+)musec.*")
def parse_flowcutter_partition_output(path):
  stats = { 'cch_ordering_running_time_s': 0.0 }

  with open(path, 'r') as f:
    for line in f:
      if not 'graph' in stats:
        stats['graph'] = path_to_graph(line.strip())
      else:
        match = runtime_pattern.match(line)
        if match:
          stats['cch_ordering_running_time_s'] += int(match[1]) / 1000000

  return stats

cch_ordering = pd.DataFrame.from_records([parse_flowcutter_partition_output(path) for path in glob.glob(base + "preprocessing/cch/*.out")])

In [6]:
table = graphs.groupby(['graph'])[['basic_customization_running_time_ms', 'contraction_running_time_ms', 'graph_build_running_time_ms', 
    'perfect_customization_running_time_ms', 'respecting_running_time_ms', 'num_nodes', 'num_edges']].mean()

table = table.reindex(['OSM Ger', 'DIMACs Europe', 'TDGer06', 'TDEur17', 'TDEur20'])

table = table.join(ch_preprocessing.groupby('graph').mean()).join(cch_ordering.groupby('graph').mean())
table['num_nodes'] = table['num_nodes'] / 1000000.0
table['num_edges'] = table['num_edges'] / 1000000.0
table['cch_phase1_s'] = table['cch_ordering_running_time_s'] + (table['contraction_running_time_ms'] / 1000)
table['cch_phase2_s'] = (table['respecting_running_time_ms'] + table['basic_customization_running_time_ms'] + table['perfect_customization_running_time_ms'] + table['graph_build_running_time_ms']) / 1000
table = table.reindex(columns=['num_nodes', 'num_edges', 'ch_contraction_running_time_s', 'cch_phase1_s', 'cch_phase2_s'])
table = table.round(1)
table

,num_nodes,num_edges,ch_contraction_running_time_s,cch_phase1_s,cch_phase2_s
graph,,,,,
OSM Ger,16.2,35.4,312.8,1466.7,10.1
DIMACs Europe,18.0,42.2,276.2,2480.9,12.4
TDGer06,4.7,10.8,59.2,331.7,2.7
TDEur17,25.8,55.5,293.9,2102.3,14.1
TDEur20,28.5,60.9,311.9,2219.5,15.2
